In [1]:
import os
os.chdir("/sasdata/corpid/a568594/playground/TM/")
import pandas as pd
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
import numpy as np
from scipy import spatial

## Word2Vec - What is it?? 

In [1]:
%%html
<header><h1 align="center">Context Window</h1></header></br>
<img src="context_window.png" width=900,height=500>

In [3]:
%%html
<header><h1 align="center">Continous Bag of Words</h1></header></br>
<img src="cbow.png" width=500,height=300>

In [4]:
%%html
<header><h1 align="center">SkipGram</h1></header></br>
<img src="skip-gram.png" width=500,height=300>

# Syntax to train a fresh corpus

In [2]:
import gensim

Using Theano backend.


In [6]:
#WordVec_model = gensim.models.Word2Vec(sentences,size=100,window=5,workers=32)

# Importing an existing trained model

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format("google_news_vectors_negative300.bin", binary=True)

In [8]:
len(model.vocab)

3000000

In [9]:
print len(model['India'])
print model['India']

300
[ -3.24707031e-02   6.10351562e-02   1.72851562e-01   2.53906250e-01
  -1.97265625e-01  -1.55273438e-01   1.06933594e-01  -2.31445312e-01
  -2.09960938e-01  -2.58789062e-02  -1.32812500e-01  -1.04492188e-01
   3.32031250e-01  -1.74804688e-01   1.45874023e-02  -7.12890625e-02
   1.62353516e-02   6.59179688e-02  -1.54418945e-02   5.12695312e-02
   7.05718994e-05  -1.47460938e-01   2.57873535e-03  -1.23046875e-01
   1.02539062e-01   1.39648438e-01  -3.88671875e-01  -1.63085938e-01
  -8.93554688e-02   7.27539062e-02  -9.66796875e-02  -2.79296875e-01
  -4.23828125e-01   4.24804688e-02   2.74658203e-02  -8.74023438e-02
  -2.17773438e-01  -5.34667969e-02   2.91015625e-01   3.44238281e-02
   8.44726562e-02   8.15429688e-02  -3.78417969e-02   2.40234375e-01
  -2.89306641e-02  -2.33154297e-02  -2.27539062e-01   7.51953125e-02
   1.80664062e-01   2.92968750e-02   5.20019531e-02   1.51367188e-01
   1.47247314e-03  -2.85644531e-02   2.56347656e-02   3.12500000e-02
  -1.76757812e-01  -1.55273438

# Vector Space and Similarity 

In [10]:
%%html
<header><h1 align="center">Cosine Similarity</h1></header></br>
<img src="cosinesimilarity.png" width=1300,height=600>

In [12]:
model.most_similar('romantic_getaway',topn=30)

[(u'romantic_getaways', 0.6873140335083008),
 (u'babymoon', 0.6238955855369568),
 (u'vacation', 0.5957752466201782),
 (u'romantic_candlelit_dinner', 0.5917630195617676),
 (u'getaways', 0.5899077653884888),
 (u'belated_honeymoon', 0.5855216979980469),
 (u'romantic_rendezvous', 0.5710945129394531),
 (u'honeymoon', 0.5697656869888306),
 (u'vacay', 0.5658637285232544),
 (u'getaway', 0.5635283589363098),
 (u'daytrip', 0.5572288036346436),
 (u'honeymooners', 0.5499558448791504),
 (u'Romance_Package', 0.5465493202209473),
 (u'Sweet_Romance_Getaway', 0.5441534519195557),
 (u'Babymoon', 0.5365304946899414),
 (u'romantic', 0.5355578660964966),
 (u'Las_Alamandas', 0.5338742733001709),
 (u'romantic_interlude', 0.5329229831695557),
 (u'Girlfriends_Getaway', 0.5295475721359253),
 (u'weekend_getaways', 0.526552140712738),
 (u'Romantic_Getaway', 0.5230082273483276),
 (u'Wakaya_Club', 0.5225156545639038),
 (u'Destination_Wedding', 0.5209423303604126),
 (u'Costa_Careyes_Mexico', 0.5194153189659119),
 (u

In [12]:
model.similarity('angelina_jolie','jennifer_aniston')

0.71678421187899066

# Extending the vectors to the sentences

In [13]:
def get_sentence_vectors(words, model, vector_len):
    featureVec = np.zeros((vector_len,), dtype="float32")
    nwords = 0

    for word in words:
        if word in model.vocab:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
            
    if (nwords>0):
        featureVec = np.divide(featureVec, nwords)
    
    return featureVec            

In [14]:
s1 = 'Modi speaks to the reporters in Karnataka'
s2 = 'The BJP greets the journalists in Bangalore'

In [15]:
def removes_stop(sentence):
    stops = [line.rstrip() for line in open("stop_words.txt", 'r')] 
    words = sentence.lower().split()
    return " ".join([w for w in words if not w in stops])

In [16]:
s1_clean = removes_stop(s1)
s2_clean = removes_stop(s2)

In [17]:
v1 = get_sentence_vectors(s1_clean.split(), model, 300)
v2 = get_sentence_vectors(s2_clean.split(), model, 300)

In [18]:
print 1 - spatial.distance.cosine(v1,v2)

0.676180677206


# How do we use this in the modelling?

In [19]:
train = pd.read_csv("labeledTrainData.tsv", header=0,delimiter="\t", quoting=3)

In [20]:
def review_to_words( raw_review ):

    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()     
    stops = [line.rstrip() for line in open("stop_words.txt", 'r')]                 
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))  

In [21]:
num_reviews = train["review"].size

clean_train_reviews = []

for i in xrange( 0, num_reviews ):
    clean_train_reviews.append( review_to_words( train["review"][i] ) )

/export/home/aaescore/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [22]:
print clean_train_reviews[0]

stuff moment mj ve listening music watching odd documentary watched wiz watched moonwalker maybe certain insight guy cool eighties maybe mind whether guilty innocent moonwalker part biography part feature film remember cinema originally released subtle messages mj feeling press obvious message drugs bad m kay visually impressive course michael jackson unless remotely mj anyway hate boring call mj egotist consenting making movie mj fans would fans true actual feature film finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted people supplying drugs etc dunno maybe hates mj music lots cool mj turning car robot whole speed demon sequence director patience saint filming kiddy bad sequence usually directors hate working kid let alone whole bunch performing complex dance scene bottom line movie people mj level people stay away wholesome message ironically mj best

In [23]:
vectors = [get_sentence_vectors(review.split(),model, 300) for review in clean_train_reviews]

In [24]:
print len(vectors), len(clean_train_reviews)

25000 25000


In [25]:
print len(vectors[0])

300


In [26]:
train_data_features = np.array(vectors)

In [27]:
train_data_features.shape

(25000, 300)

# Back to our old friends - The classifier and the cluster

In [28]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["sentiment"] )

In [29]:
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

print test.shape

num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

(25000, 2)
Cleaning and parsing the test set movie reviews...



In [30]:
print "Vectorizing the test set movie reviews...\n"
vectors_test = [get_sentence_vectors(review.split(),model, 300) for review in clean_test_reviews]
test_data_features = np.array(vectors_test)

Vectorizing the test set movie reviews...



In [31]:
print "Prediciting the class and printing it to a file...\n"

result = forest.predict(test_data_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

Prediciting the class and printing it to a file...



In [34]:
unl_train = pd.read_csv("unlabeledTrainData.tsv", header=0,delimiter="\t", quoting=3)
print unl_train.shape

unl_clean_train_reviews = []
unl_num_reviews = unl_train["review"].size

for i in xrange( 0, unl_num_reviews ):
    unl_clean_train_reviews.append( review_to_words( unl_train["review"][i] ))

vectors_unl = [get_sentence_vectors(review.split(),model, 300) for review in unl_clean_train_reviews]
unl_data_features = np.array(vectors_unl)

(50000, 2)


# Exercises to be tried
## - Try creating clusters using unl_data_features using kmeans
## - Can you try changing the distance metric from 'Eucledian' to 'Cosine'? 

# ------------------------------------------------------------------------------------------------------------